In [ ]:
import base
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

sns.set_style('ticks')
sns.set_context('talk',rc={'font.family': 'sans-serif', 'font.sans-serif':['Helvetica Neue']})

## Setup

- Load data
- Add metadata
- Draw gates
- Gate transfected cells

Result from this section: DataFrame `df` representing transfected cells.

Load exp93, which compares single-transcript, dual-transcript, and dual-vector implementations

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'
exp93_path = base_path/'2024.04.14_exp93'/'export'

plates = pd.DataFrame({
    'data_path': [exp93_path/'plate1', exp93_path/'plate2', exp93_path/'plate3',],
    'yaml_path': [exp93_path/'exp93_wells.yaml']*3,
    'biorep': [1, 2, 3],
    'exp': ['exp93']*3,
})

output_path = rd.rootdir/'output'/'two-gene'/'more'
cache_path = rd.rootdir/'output'/'two-gene'/'data.gzip'

metadata_keys = set()
for p in plates['yaml_path'].unique():
    rd.plot.plot_well_metadata(p)
    metadata_keys.update(rd.flow.load_well_metadata(p).keys())
display(metadata_keys)

In [ ]:
# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mRuby2-A','FSC-A','SSC-A','SNAP-647-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))
display(data)

In [ ]:
# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')

# Rename far-red channel
data.rename(columns={'SNAP-647-A': 'iRFP-A'}, inplace=True)

# Add metadata for second construct
metadata_construct2 = pd.read_excel(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct2-metadata.xlsx')
display(metadata_construct2)
data = data.merge(metadata_construct2, how='left', on='construct2')

data['condition'] = data['construct'] + '_' + data['construct2']
display(data)

In [ ]:
metadata_comb = data.drop_duplicates('condition')[['construct','construct2','condition']]
metadata_comb = metadata_comb.merge(metadata, how='left', on='construct')
metadata_comb = metadata_comb.merge(metadata_construct2, how='left', on='construct2')

# Create color palette
metadata_comb.loc[metadata_comb['design']==1, 'color'] = base.colors['teal']
metadata_comb.loc[metadata_comb['design']==2, 'color'] = base.colors['orange']
metadata_comb.loc[metadata_comb['design']==3, 'color'] = base.colors['red']
metadata_comb.loc[metadata_comb['gene']=='2T', 'color'] = base.colors['green']
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='EF1a'), 'color'] = base.colors['purple']
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='U6'), 'color'] = base.colors['blue']
metadata_comb.loc[(metadata_comb['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata_comb.loc[((metadata_comb['design']==1) & (metadata_comb['ts_kind']=='NT')), 'color'] = metadata_comb.loc[((metadata_comb['design']==1) & (metadata_comb['ts_kind']=='NT')), 'color'].apply(base.get_light_color)
metadata_comb.loc[((metadata_comb['design']==3) & (metadata_comb['ts_kind']=='NT')), 'color'] = metadata_comb.loc[((metadata_comb['design']==3) & (metadata_comb['ts_kind']=='NT')), 'color'].apply(base.get_dark_color)
metadata_comb.loc[metadata_comb['group'].isin(['base','marker']), 'color'] = 'black'

# markers
metadata_comb['markers'] = 'X'
metadata_comb.loc[metadata_comb['gene']=='1T', 'markers'] = 'o'
metadata_comb.loc[metadata_comb['gene']=='2T', 'markers'] = 'D'
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='EF1a'), 'markers'] = '^'
metadata_comb.loc[(metadata_comb['gene']=='2V') & (metadata_comb['construct2_promoter']=='U6'), 'markers'] = 'v'

metadata_comb['condition'] = metadata_comb['construct'] + '_' + metadata_comb['construct2']
display(metadata_comb)

metadata_dict = metadata_comb.set_index('condition').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

In [ ]:
metadata_comb.loc[(metadata_comb['ts_loc']=='5\''), 'color'] = metadata_comb.loc[(metadata_comb['ts_loc']=='5\''), 'color'].apply(base.get_light_color)
metadata_dict = metadata_comb.set_index('condition').to_dict('dict')
main_palette2 = metadata_dict['color']

Draw gates based on untransfected population, then gate transfected cells

In [ ]:
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='GEEC555'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Indicate which channels are relevant for each experiment
gates['marker'] = 'iRFP-A'
gates['output'] = 'mRuby2-A'

# Add missing iRFP gate
gate_iRFP = 2.5e2
g = sns.displot(data=data, x='iRFP-A', col='biorep', hue='condition', palette=main_palette,
                 kind='kde', fill=False, legend=False, common_norm=True, log_scale=True)
for name, ax in g.axes_dict.items():
    ax.axvline(gate_iRFP, color='black')
g.figure.savefig(rd.outfile(output_path/(f'hist_gate_iRFP.svg')), bbox_inches='tight')

gates['iRFP-A'] = [gate_iRFP]

display(gates)

Gate data per experiment based on transfection marker expression

In [ ]:
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]
display(df)

In [ ]:
df['marker'] = df['marker'].astype(float)
df['output'] = df['output'].astype(float)

# Group and compute stats
stat_list = [np.mean, np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df.groupby(by=['condition','construct','construct2','biorep','exp'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['marker'].count().reset_index()['marker']
stats = stats.merge(metadata, how='left', on='construct')
stats.sort_values(['design','ts_kind','ts_num'], inplace=True)
stats = stats.merge(metadata_construct2, how='left', on='construct2')

stats['condition'] = stats['construct'] + '_' + stats['construct2']
display(stats)

stat_name = {'output_gmean': 'Geometric\nmean', 'output_std': 'Standard\ndeviation', 'output_variation': 'Coefficient\nof variation'}

In [ ]:
# Bin by marker quantiles
by = ['construct2','construct','exp','biorep']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])
display(df_quantiles)

In [ ]:
# Quantile stats & slope
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','construct2','biorep','exp','bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
display(stats_quantiles)

stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)

In [ ]:
# Compute slope for all constructs
def get_slope(df):
    slope, intercept, r_value, p_value, stderr = sp.stats.linregress(df['bin_marker_quantiles_median_log'], df['output_gmean_log'])
    result = pd.DataFrame(columns=['slope', 'intercept_log', 'r_value', 'p_value', 'stderr'])
    result.loc[len(result.index)] = [slope, intercept, r_value, p_value, stderr]
    return result

fits = stats_quantiles.groupby(['construct','construct2','exp','biorep'])[stats_quantiles.columns].apply(get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')
fits.sort_values(['design','ts_kind','ts_num'], inplace=True)
fits = fits.merge(metadata_construct2, how='left', on='construct2')

fits['condition'] = fits['construct'] + '_' + fits['construct2']
display(fits)

### Distribution plots

In [ ]:
# single-transcript design 1
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['gene']=='1T') & (df_quantiles['design']==1) & (df_quantiles['group']=='controller')) | ((df_quantiles['gene']=='1T') & (df_quantiles['group']=='base'))) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_single-transcript_design1.svg')), bbox_inches='tight')

In [ ]:
# single-transcript designs 2&3
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['gene']=='1T') & (df_quantiles['design']>1) & (df_quantiles['group']=='controller')) | ((df_quantiles['gene']=='1T') & (df_quantiles['group']=='base'))) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_single-transcript_designs2-3.svg')), bbox_inches='tight')

In [ ]:
# dual-transcript
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[((df_quantiles['gene']=='2T') & (df_quantiles['group']=='dual')) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette2, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette2, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_dual-transcript.svg')), bbox_inches='tight')

In [ ]:
# dual-transcript
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[((df_quantiles['gene']=='2T') & (df_quantiles['group']=='dual') & (df_quantiles['ts_loc']=='3\'')) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_dual-transcript_ts3.svg')), bbox_inches='tight')

In [ ]:
# dual-transcript
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[((df_quantiles['gene']=='2T') & (df_quantiles['group']=='dual') & (df_quantiles['ts_loc']=='5\'')) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_dual-transcript_ts5.svg')), bbox_inches='tight')

In [ ]:
# dual-vector EF1a
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[((df_quantiles['gene']=='2V') & (df_quantiles['construct2_promoter']=='EF1a')) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette2, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette2, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_dual-vector-EF1a.svg')), bbox_inches='tight')

In [ ]:
# dual-vector EF1a
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[((df_quantiles['gene']=='2V') & (df_quantiles['construct2_promoter']=='EF1a') & (df_quantiles['group'].isin(['ts3','base']))) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_dual-vector-EF1a_ts3.svg')), bbox_inches='tight')

In [ ]:
# dual-vector EF1a
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[((df_quantiles['gene']=='2V') & (df_quantiles['construct2_promoter']=='EF1a') & (df_quantiles['group'].isin(['ts5','base']))) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_dual-vector-EF1a_ts5.svg')), bbox_inches='tight')

In [ ]:
# dual-vector U6
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[((df_quantiles['gene']=='2V') & (df_quantiles['construct2_promoter']=='U6')) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette2, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette2, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_dual-vector-U6.svg')), bbox_inches='tight')

In [ ]:
# dual-vector U6
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[((df_quantiles['gene']=='2V') & (df_quantiles['construct2_promoter']=='U6') & (df_quantiles['group'].isin(['ts3','base']))) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_dual-vector-U6_ts3.svg')), bbox_inches='tight')

In [ ]:
# dual-vector U6
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[((df_quantiles['gene']=='2V') & (df_quantiles['construct2_promoter']=='U6') & (df_quantiles['group'].isin(['ts5','base']))) &
                       (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='condition', palette=main_palette, 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_dual-vector-U6_ts5.svg')), bbox_inches='tight')

### Stat plots

In [ ]:
stats['group'].unique()

In [ ]:
# all architectures with 3'UTR target sites
fig, axes = plt.subplots(1,3, figsize=(15,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.2))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = stats[((stats['gene']=='1T') & (stats['design']==1) & (stats['group']=='controller')) |
                ((stats['gene']=='2T') & (stats['group']=='dual') & (stats['ts_loc']=='3\'')) |
                ((stats['gene']=='2V') & (stats['group']=='ts3')) |
                (stats['group']=='base')].copy()
plot_df.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)
plot_df2 = fits[((fits['gene']=='1T') & (fits['design']==1) & (fits['group']=='controller')) |
                ((fits['gene']=='2T') & (fits['group']=='dual') & (fits['ts_loc']=='3\'')) |
                ((fits['gene']=='2V') & (fits['group']=='ts3')) |
                (fits['group']=='base')].copy()
plot_df2.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='output_gmean', hue='condition', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean\n', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['ts_kind'], ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='output_std', hue='condition', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation\n', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['ts_kind'], ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='slope', hue='condition', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope\n', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',)
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['ts_kind'], ax=ax)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# for i,ax in enumerate(axes):
#     ax.axvspan(2.5, 4.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
#     ax.axvspan(7.5, 10.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
#     ax.annotate('1T', (1,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=sns.plotting_context('talk')['legend.fontsize'])
#     ax.annotate('2T', (3.5,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=sns.plotting_context('talk')['legend.fontsize'])
#     ax.annotate('2V\nEF1a', (6,ax.get_ylim()[1]), ha='center', va='center', ma='center', fontsize=sns.plotting_context('talk')['legend.fontsize'])
#     ax.annotate('2V\nU6', (9,ax.get_ylim()[1]), ha='center', va='center', ma='center', fontsize=sns.plotting_context('talk')['legend.fontsize'])

fig.savefig(rd.outfile(output_path/(f'stat_ts3.svg')), bbox_inches='tight')

In [ ]:
# all architectures with 5'UTR target sites
fig, axes = plt.subplots(1,3, figsize=(15,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.2))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = stats[((stats['gene']=='1T') & (stats['design']>1) & (stats['group']=='controller')) |
                ((stats['gene']=='2T') & (stats['group']=='dual') & (stats['ts_loc']=='5\'')) |
                ((stats['gene']=='2V') & (stats['group']=='ts5')) |
                (stats['group']=='base')].copy()
plot_df.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)
plot_df2 = fits[((fits['gene']=='1T') & (fits['design']>1) & (fits['group']=='controller')) |
                ((fits['gene']=='2T') & (fits['group']=='dual') & (fits['ts_loc']=='5\'')) |
                ((fits['gene']=='2V') & (fits['group']=='ts5')) |
                (fits['group']=='base')].copy()
plot_df2.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='output_gmean', hue='condition', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean\n', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['ts_kind'], ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='output_std', hue='condition', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation\n', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['ts_kind'], ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='slope', hue='condition', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope\n', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=(0,1.3), ylabel='',)
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['ts_kind'], ax=ax)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# for i,ax in enumerate(axes):
#     ax.axvspan(4.5, 6.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
#     ax.axvspan(9.5, 13.5, color=base.get_light_color(base.colors['gray']), alpha=0.2,)
#     ax.annotate('1T', (2,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=sns.plotting_context('talk')['legend.fontsize'])
#     ax.annotate('2T', (5.5,ax.get_ylim()[1]), ha='center', va='bottom', ma='center', fontsize=sns.plotting_context('talk')['legend.fontsize'])
#     ax.annotate('2V\nEF1a', (8,ax.get_ylim()[1]), ha='center', va='center', ma='center', fontsize=sns.plotting_context('talk')['legend.fontsize'])
#     ax.annotate('2V\nU6', (11,ax.get_ylim()[1]), ha='center', va='center', ma='center', fontsize=sns.plotting_context('talk')['legend.fontsize'])

fig.savefig(rd.outfile(output_path/(f'stat_ts5.svg')), bbox_inches='tight')

### CDF plots

In [ ]:
df_quantiles['gene2'] = df_quantiles['gene'] + '_' + df_quantiles['construct2_promoter']

In [ ]:
xlim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['gene']=='1T') & (df_quantiles['design']==1) & (df_quantiles['group']=='controller')) |
                        ((df_quantiles['gene']=='2T') & (df_quantiles['group']=='dual') & (df_quantiles['ts_loc']=='3\'')) |
                        ((df_quantiles['gene']=='2V') & (df_quantiles['group']=='ts3')) |
                        (df_quantiles['group']=='base')) & (df_quantiles['biorep']==biorep)].copy()
plot_df.sort_values(['gene','construct2_promoter'], inplace=True)
g = sns.displot(data=plot_df, x='output', col='gene2',
                hue='condition', palette=main_palette, kind='kde',
                cumulative=True, common_norm=False, legend=False, log_scale=True)
for gene2, ax in g.axes_dict.items():
    sns.kdeplot(data=df_quantiles[df_quantiles['group']=='marker'], x='output', hue='condition', palette=main_palette,
                cumulative=True, common_norm=False, legend=False, log_scale=True, ax=ax, ls=':', zorder=0)
    ax.grid(zorder=-1)
    ax.set(title='')

g.figure.savefig(rd.outfile(output_path/(f'cdf_ts3.svg')), bbox_inches='tight')

In [ ]:
xlim = (2e1,1e5)
biorep = 2
plot_df = df_quantiles[(((df_quantiles['gene']=='1T') & (df_quantiles['design']>1) & (df_quantiles['group']=='controller')) |
                        ((df_quantiles['gene']=='2T') & (df_quantiles['group']=='dual') & (df_quantiles['ts_loc']=='5\'')) |
                        ((df_quantiles['gene']=='2V') & (df_quantiles['group']=='ts5')) |
                        (df_quantiles['group']=='base')) & (df_quantiles['biorep']==biorep)].copy()
plot_df.sort_values(['gene','construct2_promoter'], inplace=True)
g = sns.displot(data=plot_df, x='output', col='gene2',
                hue='condition', palette=main_palette, kind='kde',
                cumulative=True, common_norm=False, legend=False, log_scale=True)
for gene2, ax in g.axes_dict.items():
    sns.kdeplot(data=df_quantiles[df_quantiles['group']=='marker'], x='output', hue='condition', palette=main_palette,
                cumulative=True, common_norm=False, legend=False, log_scale=True, ax=ax, ls=':', zorder=0)
    ax.grid(zorder=-1)
    ax.set(title='')

g.figure.savefig(rd.outfile(output_path/(f'cdf_ts5.svg')), bbox_inches='tight')

### Explore outlier point

In [ ]:
# all architectures with 5'UTR target sites
fig, axes = plt.subplots(1,3, figsize=(15,5), gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.2))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = stats[(stats['group']=='base')].copy()
plot_df.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)
plot_df2 = fits[(fits['group']=='base')].copy()
plot_df2.sort_values(['gene','construct2_promoter','group','ts_kind'], inplace=True)

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='output_gmean', hue='condition', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean\n', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['gene'], ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='output_std', hue='condition', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation\n', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['gene'], ax=ax)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('condition', sort=False):
    sns.stripplot(data=group, x='condition', y='slope', hue='condition', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=9, jitter=0.2, linewidth=1, edgecolor='white')

ax.set(title='Slope\n', xlim=(-0.5, len(plot_df['condition'].unique())-0.5), xlabel='', ylim=(0,1.3), ylabel='',)
rd.plot.generate_xticklabels(metadata_comb, 'condition', ['gene'], ax=ax)
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path/(f'stat_ts5.svg')), bbox_inches='tight')

In [ ]:
# Bioreps of 1T base gene
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = df_quantiles[(df_quantiles['gene']=='1T') & (df_quantiles['group']=='base')]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='biorep', 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='biorep', 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_1T-base.svg')), bbox_inches='tight')

In [ ]:
# Wells of 1T base gene, biorep 1
fig, axes = plt.subplots(1,2, figsize=(8,5), sharey=True, gridspec_kw=dict(width_ratios=(1,0.4)))
xlim = (2e2,2e4)
ylim = (2e1,1e5)
biorep = 1
plot_df = df_quantiles[(df_quantiles['gene']=='1T') & (df_quantiles['group']=='base') & (df_quantiles['biorep']==biorep)]

# line plot
ax = axes[0]
sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='well', 
             legend=False, dashes=False, style='condition', markers=main_markers, ax=ax,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlim=xlim, ylim=ylim, xlabel='marker',)
sns.despine(ax=ax)
marker_baseline = sp.stats.gmean(df_quantiles.loc[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_on()
ax.annotate('marker only', (ax.get_xlim()[1], marker_baseline), ha='right', va='bottom', fontsize=sns.plotting_context('talk')['legend.fontsize'])

# histogram
ax = axes[1]
sns.kdeplot(data=plot_df, y='output', hue='well', 
             legend=False, log_scale=True, common_norm=False, ax=ax)
sns.kdeplot(data=df_quantiles[(df_quantiles['group']=='marker') & (df_quantiles['biorep']==biorep)], y='output', hue='condition', palette=main_palette, 
             legend=False, log_scale=True, common_norm=False, ax=ax, ls=':', zorder=0)
sns.despine(ax=ax)
ax.set(xlabel='density',)

fig.savefig(rd.outfile(output_path/(f'line-marg-hist_1T-base-biorep1.svg')), bbox_inches='tight')

In [ ]:
display(df_quantiles['gene'].unique())

In [ ]:
# Joint plot
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = df_quantiles[(df_quantiles['gene']=='1T') & (df_quantiles['group']=='base')]

g = sns.jointplot(data=plot_df, x='marker', y='output', hue='biorep',
                  kind='kde', log_scale=True, common_norm=False, levels=7)

In [ ]:
# Joint plot
xlim = (2e2,2e4)
ylim = (2e1,1e5)
plot_df = df_quantiles[(df_quantiles['group']=='base')]

g = sns.displot(data=plot_df, x='marker', y='output', hue='biorep', col='condition',
                  kind='kde', log_scale=True, common_norm=False, levels=7)